* [#4](https://github.com/salgo60/SCB-Wikidata/issues/4)
* denna [SCB4_linkroot_all.ipynb](https://github.com/salgo60/SCB-Wikidata/blob/main/notebook/SCB4_linkroot_all.ipynb)

Tanken är att kolla alla Wikipedia domäner

In [79]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-11-17 09:47:55


In [66]:
def read_language_codes_auto():
    """
    Load all known Wikipedia language codes.
    """
    url = "https://meta.wikimedia.org/wiki/Special:SiteMatrix?uselang=en"
    from bs4 import BeautifulSoup
    import requests

    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    codes = set()
    for link in soup.select("a"):
        href = link.get("href", "")
        if href.startswith("//") and ".wikipedia.org" in href:
            lang = href.split("//")[1].split(".")[0]
            codes.add(lang)

    return sorted(codes)


In [36]:
    
def read_domains(file_path): 
    print(f"[DEBUG] Reading domains from: {file_path}") 
    df = pd.read_csv(file_path, header=None, names=['domain']) 
    domains_list = df['domain'].dropna().unique().tolist() 
    print(f"[DEBUG] Found {len(domains_list)} domains.") 
    return domains_list

In [70]:
import os  
CHECKPOINT_LINKS_ALL  = "results_all/_checkpoint_exturlusage_all.csv"
CHECKPOINT_STATUS_ALL = "results_all/_checkpoint_status_all.csv"
# -------------------------
# CONFIG — justera vid behov
# -------------------------
CONFIG = {
    "checkpoint_links_all": "results_all/_checkpoint_exturlusage_all.csv",
    "checkpoint_status_all": "results_all/_checkpoint_status_all.csv",
    "out_all_links": "results_all/all_links.csv",
}

os.makedirs("results", exist_ok=True)

In [71]:
# -----------------------------------------------------------
# Fetch exturlusage entries for one lang/domain
# -----------------------------------------------------------
def fetch_exturlusage(lang, domain):
    base = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "list": "exturlusage",
        "euquery": domain,
        "eulimit": "max"
    }
    while True:
        r = session.get(base, params=params, timeout=10)
        data = r.json()
        for item in data.get("query", {}).get("exturlusage", []):
            yield {
                "lang": lang,
                "page_title": item.get("title"),
                "url": item.get("url"),
                "wiki_link": f"https://{lang}.wikipedia.org/wiki/{item.get('title').replace(' ', '_')}"
            }

        if "continue" not in data:
            break
        params.update(data["continue"])
        time.sleep(0.3)
# -----------------------------------------------------------
# Modified main() with resume support
# -----------------------------------------------------------


In [72]:
from concurrent.futures import ThreadPoolExecutor, as_completed

MAX_WORKERS             = 5
SLEEP_BETWEEN_REQUESTS  = 1
USER_CONTRIB_LIMIT      = 10


In [77]:
import os
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed

MAIN_RESULTS_OUTPUT_ALL = "results_all/results_all.csv"
WIKI_VERSIONS_FILE_ALL = "sources/wiki_versions_all.csv"
DOMAINS_FILE = "sources/domains.csv"

MAX_WORKERS = 20


def read_domains(file_path):
    print(f"[DEBUG] Reading domains from: {file_path}")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Missing file: {file_path}")
    df = pd.read_csv(file_path)
    return df['domain'].dropna().unique().tolist()


def process_lang_domain_pair(lang, domain, writer):
    """
    Dummy example of URL checking.
    Replace with your real logic.
    """
    url = f"https://{lang}.{domain}"
    print(f"[DEBUG] Checking URL: {url}")

    try:
        import requests
        r = requests.get(url, timeout=5)
        status = "OK" if r.status_code == 200 else r.status_code
    except Exception as e:
        status = f"ERROR: {e}"

    writer.writerow({
        "lang": lang,
        "domain": domain,
        "url": url,
        "page_title": "",
        "wiki_link": "",
        "user": "",
        "timestamp": status
    })


def main():

    os.makedirs("results_all", exist_ok=True)

    # Will the CSV need a header?
    write_header = not os.path.exists(MAIN_RESULTS_OUTPUT_ALL)

    main_fields = ['lang', 'domain', 'url', 'page_title',
                   'wiki_link', 'user', 'timestamp']

    with open(MAIN_RESULTS_OUTPUT_ALL, 'a', newline='', encoding='utf-8') as main_out:

        writer = csv.DictWriter(main_out, fieldnames=main_fields)
        if write_header:
            writer.writeheader()

        # Read language codes & domains
        language_codes = read_language_codes_auto()
        domains = read_domains(DOMAINS_FILE)

        print(domains)

        print("[DEBUG] Starting concurrency for (lang, domain) pairs...")

        futures = []
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            for lang in language_codes:
                for domain in domains:
                    futures.append(executor.submit(
                        process_lang_domain_pair, lang, domain, writer
                    ))

            for fut in as_completed(futures):
                try:
                    fut.result()
                except Exception as e:
                    print("[ERROR] Worker error:", e)

    print(f"[DEBUG] Done collecting main results in {MAIN_RESULTS_OUTPUT_ALL}")
    print("[DEBUG] All (lang, domain) pairs processed.")


if __name__ == "__main__":
    main()


[DEBUG] Reading domains from: sources/domains.csv


/var/folders/fd/md6r13sj0wsbg_6_xl160d300000gn/T/ipykernel_66257/3932524435.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(r.text, "html.parser")


KeyError: 'domain'

In [34]:
import os
import time 
import pandas as pd
from urllib.parse import urlparse

# -------------------------------------------------
# LADDA LÄNKAR
# -------------------------------------------------
df_all = pd.read_csv(CHECKPOINT_LINKS_ALL)

print("Alla rader i checkpoint:", len(df_all))

# Ta unika URL:er
urls = df_all["url"].dropna().unique().tolist()
total = len(urls)
print(f"Totalt unika URL: {total}")

# -------------------------------------------------
# LÄNKAR PER WIKIPEDIA-SPRÅK
# -------------------------------------------------

# Försök hitta vilken kolumn som innehåller språkkoden
possible_lang_cols = ["lang", "language", "wiki", "wiki_lang", "language_code"]
lang_col = None

for c in possible_lang_cols:
    if c in df_all.columns:
        lang_col = c
        break

if lang_col is None:
    print("⚠️ Hittar ingen språk-kolumn i datan! Inga språk-stats kan visas.")
else:
    print(f"\nSpråk-kolumn hittad: '{lang_col}'")

    # Totalt antal länkar per språk (inkl. dubbletter)
    print("\n--- Statistik per språk (alla rader) ---")
    lang_counts_all = df_all[lang_col].value_counts()
    for lang, cnt in lang_counts_all.items():
        print(f"{lang:5} {cnt}")

    # Antal UNIKA URLer per språk
    print("\n--- Statistik per språk (unika URL:er) ---")
    df_all_unique = df_all.drop_duplicates(subset=["url"])
    lang_counts_unique_all = df_all_unique[lang_col].value_counts()
    for lang, cnt in lang_counts_unique_all.items():
        print(f"{lang:5} {cnt}")

    print("\nAntal språk:", len(lang_counts_all))

# -------------------------------------------------
# DOMÄNSTATISTIK
# -------------------------------------------------
domains = [urlparse(u).netloc.lower() for u in urls]
domain_counts = pd.Series(domains).value_counts()

print("\n--- Statistik per domän (flest först) ---")
for dom, cnt in domain_counts.items():
    print(f"{dom:35} {cnt}")

print("\nAntal domäner:", len(domain_counts))

# -------------------------------------------------
# LADDA PREVIOUS STATUS (resume)
# -------------------------------------------------
status_map = {}
if os.path.exists(CHECKPOINT_STATUS_ALL):
    prev_all = pd.read_csv(CHECKPOINT_STATUS_ALL)
    status_map_all = dict(zip(prev_all["url"], prev_all["status"]))
    print("\nLaddade tidigare status:", len(status_map))

# Filtrera bort redan testade länkar
remaining = [u for u in urls if u not in status_map]

print("\n--- Sammanfattning ---")
print(f"Totalt unika URL:         {total}")
print(f"Antal domäner:             {len(domain_counts)}")

# Visa topp 10 domäner
print("\nTopp 100 domäner:")
print(domain_counts.head(100))



Alla rader i checkpoint: 0
Totalt unika URL: 0

Språk-kolumn hittad: 'lang'

--- Statistik per språk (alla rader) ---

--- Statistik per språk (unika URL:er) ---

Antal språk: 0

--- Statistik per domän (flest först) ---

Antal domäner: 0

Laddade tidigare status: 0

--- Sammanfattning ---
Totalt unika URL:         0
Antal domäner:             0

Topp 100 domäner:
Series([], Name: count, dtype: int64)


In [27]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-11-16 11:11:24
Total time elapsed: 08 minutes 19.46 seconds
